# A simple waveform demo

Select a tool on the left to navigate the waveform and click a button to play a portion of the waveform in the browser.

If running in a BinderHub instance instead of in a local notebook, it might be necessary to change the `default_url` value below.

In [ ]:
import os
import urllib
from bokeh_phon.models.audio_plot import AudioPlot
from bokeh.models import Button
from bokeh.io import show, output_notebook
from bokeh.layouts import column
import parselmouth
import numpy as np
output_notebook()

In [ ]:
# The remote_jupyter_proxy_url function is required when running on a BinderHub instance.
# Change the default_url value to match the hostname of your instance after it has
# started. The current value is the most frequent result when launching from mybinder.org.
default_url = 'https://hub.gke.mybinder.org/'

def remote_jupyter_proxy_url(port):
    """
    Callable to configure Bokeh's show method when a proxy must be
    configured.

    If port is None we're asking about the URL
    for the origin header.
    """
    try:
        base_url = os.environ['EXTERNAL_URL']
    except KeyError:
        base_url = default_url
    host = urllib.parse.urlparse(base_url).netloc

    # If port is None we're asking for the URL origin
    # so return the public hostname.
    if port is None:
        return host

    service_url_path = os.environ['JUPYTERHUB_SERVICE_PREFIX']
    proxy_url_path = 'proxy/%d' % port

    user_url = urllib.parse.urljoin(base_url, service_url_path)
    full_url = urllib.parse.urljoin(user_url, proxy_url_path)
    return full_url

In [ ]:
def myapp(doc):
    snd = parselmouth.Sound('../resource/the_north_wind_and_the_sun.reverse.wav')
    ap = AudioPlot(
        samples=np.squeeze(snd.values),
        fs=snd.sampling_frequency,
        # Remaining arguments are passed to Figure().
        plot_height=200,
        toolbar_location='left'
    )
    playallbtn = Button(label='Play all')
    playallbtn.js_on_event('button_click', ap.js_playall_cb)
    playselbtn = Button(label='Play selection')
    playselbtn.js_on_event('button_click', ap.js_playsel_cb)
    playxrbtn = Button(label='Play x range')
    playxrbtn.js_on_event('button_click', ap.js_playxr_cb)
    col = column(playallbtn, playselbtn, playxrbtn, ap)
    doc.add_root(col)
    return doc


# The notebook_url parameter is required when running in a BinderHub instance.
# If running a local notebook, omit that parameter.
#show(myapp)    # For running a local notebook
show(myapp, notebook_url=remote_jupyter_proxy_url)